# Organize Games from json files

In [1]:
import json
import numpy as np
from collections import OrderedDict
from pathlib import Path

In [2]:
root = Path().absolute().parent
root

WindowsPath('c:/Users/etien/PyProjects/ai2/chris')

In [4]:
p2ai2_all = root / 'config/ai2/ai2-games-all.json'
p2ai2_current = root / 'config/ai2/ai2-games-all-2024-07.json'
p2ai2 = root / 'config/ai2/ai2-games.json'
p2jam = root / 'config/jam/games.json'
p2zmack = root / 'config\zmack\games.json'
p2default = root / 'config/games.json'
assert p2ai2_all.is_file()
assert p2ai2_current.is_file()
assert p2ai2.is_file()
assert p2jam.is_file()
assert p2zmack.is_file()
assert p2default.is_file()

## Sort games alphabetically

In [14]:
def load_and_sort(p2json):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
    names = np.array([g['name'] for g in games.values()])
    sorted_game_idxs = np.argsort(names)
    sorted_game_keys = np.array(list(games.keys()))[sorted_game_idxs]
    sorted_list_game_dict = [games[k] for k in sorted_game_keys]
    
    return games, names, sorted_game_idxs, sorted_game_keys, sorted_list_game_dict

def sort_json_by_game_names(p2json):
    odict = OrderedDict()
    games, names, sgi, sgk, slgd = load_and_sort(p2json)
    for i, d in enumerate(slgd):
        odict[str(i+1)] = d

    print(f"{i+1} games")

    with open(p2json, 'w') as fp:
        json.dump(odict, fp, indent=4)

In [15]:
sort_json_by_game_names(p2ai2_all)
sort_json_by_game_names(p2ai2_current)
sort_json_by_game_names(p2ai2)

25 games
25 games
23 games


## Add fields in json

In [30]:
def add_field(p2json, field, save_json=False):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
        for k, v in games.items():
            game_dict = v
            game_dict[field] = ""
            games[k] = game_dict
    
    if save_json:
        with open(p2json, 'w') as fp:
            json.dump(games, fp, indent=4)

    return games

def remove_field(p2json, field, save_json=False):
    with open(p2json, 'r') as fp:
        games = json.load(fp)
        for k, v in games.items():
            game_dict = v
            if field in game_dict.keys():
                del game_dict[field]
                games[k] = game_dict
    
    if save_json:
        with open(p2json, 'w') as fp:
            json.dump(games, fp, indent=4)

    return games

In [35]:
for p2json in [p2ai2_all, p2ai2, p2jam, p2zmack, p2default]:
    add_field(p2json, 'category', save_json=True);